<a href="https://colab.research.google.com/github/MeGAMIndD/Covid_19/blob/master/epidemic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import folium
import os

import warnings
warnings.filterwarnings('ignore')
from urllib.request import urlopen

import plotly.graph_objects as go

In [0]:
pd.set_option('display.max_rows',20000, 'display.max_columns',100)

In [0]:
df_corona_in_india = pd.read_csv("/content/drive/My Drive/epidemic_modelling/covid_19_india.csv")
df_corona_india = pd.read_csv("/content/drive/My Drive/epidemic_modelling/complete.csv")
df_ICMR = pd.read_csv("/content/drive/My Drive/epidemic_modelling/ICMRTestingDetails.csv")
df_Individual = pd.read_csv("/content/drive/My Drive/epidemic_modelling/IndividualDetails.csv")
df_Hospital = pd.read_csv("/content/drive/My Drive/epidemic_modelling/HospitalBedsIndia.csv")
df_Age = pd.read_csv("/content/drive/My Drive/epidemic_modelling/AgeGroupDetails.csv")
df_Italy = pd.read_csv("/content/drive/My Drive/epidemic_modelling/covid19_italy_region.csv")
df_daily_US = pd.read_csv("/content/drive/My Drive/epidemic_modelling/us_covid19_daily.csv")
df_corona_in_india.head()

In [0]:
#Total cases of carona in India
df_corona_in_india['Total Cases'] = df_corona_in_india['Cured'] + df_corona_in_india['Deaths'] + df_corona_in_india['Confirmed']
#Active cases of carona in India
df_corona_in_india['Active Cases'] = df_corona_in_india['Total Cases'] - df_corona_in_india['Cured'] - df_corona_in_india['Deaths']
df_corona_in_india.head()

In [0]:
#Till 8th April Cases in India
df1= df_corona_in_india[df_corona_in_india['Date']=='08/04/20']
fig = px.bar(df1, x='State/UnionTerritory', y='Total Cases', color='Total Cases', height=600)
fig.update_layout(
    title='Till 8th April Total Cases in India')
fig.show()

In [0]:
#Till 8th April Active Cases in India
df1= df_corona_in_india[df_corona_in_india['Date']=='08/04/20']
fig = px.bar(df1, x='State/UnionTerritory', y='Active Cases', color='Active Cases',barmode='group', height=600)
fig.update_layout(
    title='Till 8th April Active Cases in India')
fig.show()

In [0]:
df_corona_in_india['Date'] =pd.to_datetime(df_corona_in_india.Date,dayfirst=True)
df_corona_in_india

In [0]:
#Daily Cases in India Datewise
corona_data = df_corona_in_india.groupby(['Date'])['Total Cases'].sum().reset_index().sort_values('Total Cases',ascending = True)
corona_data['Daily Cases'] = corona_data['Total Cases'].sub(corona_data['Total Cases'].shift())
corona_data['Daily Cases'].iloc[0] = corona_data['Total Cases'].iloc[0]
corona_data['Daily Cases'] = corona_data['Daily Cases'].astype(int)
fig = px.bar(corona_data, y='Daily Cases', x='Date',hover_data =['Daily Cases'], color='Daily Cases', height=500)
fig.update_layout(
    title='Daily Cases in India Datewise')
fig.show()

In [0]:
corona_data['Corona Growth Rate'] = corona_data['Total Cases'].pct_change().mul(100).round(2)
#Corona Growth Rate Comparison with Previous Day
fig = px.bar(corona_data, y='Corona Growth Rate', x='Date',hover_data =['Corona Growth Rate','Total Cases'], height=500)
fig.update_layout(
    title='Corona Growth Rate(in Percentage) Comparison with Previous Day')
fig.show()

In [0]:
#Moratality Rate
corona_data = df_corona_in_india.groupby(['Date'])['Total Cases','Active Cases','Deaths'].sum().reset_index().sort_values('Date',ascending=False)
corona_data['Mortality Rate'] = ((corona_data['Deaths']/corona_data['Total Cases'])*100)
fig = go.Figure()
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Mortality Rate'],
                    mode='lines+markers',name='Cases'))
fig.update_layout(title_text='COVID-19 Mortality Rate in INDIA',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
#DAILY DEATHS IN INDIA
corona_data = df_corona_in_india.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Deaths',ascending = True)
corona_data['DAILY DEATHS'] = corona_data['Deaths'].sub(corona_data['Deaths'].shift())
corona_data['DAILY DEATHS'].iloc[0] = corona_data['Deaths'].iloc[0]
corona_data['DAILY DEATHS'] = corona_data['DAILY DEATHS'].astype(int)
fig = px.bar(corona_data, y='DAILY DEATHS', x='Date',hover_data =['DAILY DEATHS'], color='DAILY DEATHS', height=500)
fig.update_layout(
    title='DAILY DEATHS IN INDIA')
fig.show()

In [0]:
#Total Cases in India State Datewise
corona_data = df_corona_in_india.groupby(['Date','State/UnionTerritory','Total Cases'])['Cured','Deaths','Active Cases'].sum().reset_index().sort_values('Total Cases',ascending = False)
fig = px.bar(corona_data, y='Total Cases', x='Date',hover_data =['State/UnionTerritory','Active Cases','Deaths','Cured'], color='Total Cases',barmode='group', height=700)
fig.update_layout(
    title='Indian States with Current Total Corona Cases')
fig.show()

In [0]:
df_corona_india.head()

In [0]:
#Pie chart visualization of states effected by caronavirus
fig = px.pie(df_Age, values='TotalCases', names='AgeGroup')
fig.update_layout(
    title='Age Group affected with COVID-19')
fig.show()

In [0]:
#Agewise Gender affected by COVID-19(There are lot of Nan Values)
fig = px.histogram(df_Individual.dropna(), x="age",color ='gender')
fig.update_layout(
    title='Agewise Gender affected by COVID-19')
fig.show()

In [0]:
#Agewise Covid-19 patients in State(There are lot of Nan Values)
fig = px.histogram(df_Individual.dropna(), x="age",color ='detected_state')
fig.update_layout(
    title='Agewise Covid-19 patients in State')
fig.show()

In [0]:
#Genderwise current status of COVID-19(There are lot of Nan Values)
fig = px.histogram(df_Individual.dropna(), x="gender",color ='current_status')
fig.update_layout(
    title='Genderwise current status of COVID-19')
fig.show()

In [0]:
#Total Cases Datewise of Foreign Nationals
corona_data = df_corona_india.groupby(['Date','Name of State / UT','Total Confirmed cases ( Foreign National )'])['Total Confirmed cases','Cured/Discharged/Migrated',].sum().reset_index().sort_values('Total Confirmed cases ( Foreign National )',ascending = True)
fig = px.bar(corona_data, y='Total Confirmed cases ( Foreign National )', x='Date',hover_data =['Name of State / UT','Total Confirmed cases','Cured/Discharged/Migrated'], color='Total Confirmed cases ( Foreign National )', height=700)
fig.update_layout(
    title='Total Cases Datewise of Foreign Nationals')
fig.show()

In [0]:
#Total Cases Datewise of Indian Nationals
corona_data = df_corona_india.groupby(['Date','Name of State / UT','Total Confirmed cases (Indian National)'])['Total Confirmed cases','Cured/Discharged/Migrated',].sum().reset_index().sort_values('Total Confirmed cases (Indian National)',ascending = True)
fig = px.bar(corona_data, y='Total Confirmed cases (Indian National)', x='Date',hover_data =['Name of State / UT','Total Confirmed cases','Cured/Discharged/Migrated'], color='Total Confirmed cases (Indian National)', height=700)
fig.update_layout(
    title='Total Cases Datewise of Indian  Nationals')
fig.show()

In [0]:
df_population_of_india = pd.read_csv("/content/drive/My Drive/epidemic_modelling/population_india_census2011.csv")
df_population_of_india

In [0]:
#Statewise Total Cases
df_new = df1.groupby(['State/UnionTerritory','Cured','Deaths','Active Cases'])['Total Cases'].sum().reset_index().sort_values('Total Cases',ascending = False)
df_new

In [0]:
#Using Merge two join two diffrent data frames and then sorting them in ascending
df_population_with_corona_case = df_population_of_india.merge(df_new, left_on='State / Union Territory', right_on='State/UnionTerritory')
df_population_with_corona_case=df_population_with_corona_case.drop(labels=['State/UnionTerritory','Sno'],axis=1)
df_population_with_corona_case=df_population_with_corona_case.sort_values('Total Cases',ascending=False)
df_population_with_corona_case

In [0]:
#Pie chart visualization of states effected by caronavirus
fig = px.pie(df_population_with_corona_case, values='Total Cases', names='State / Union Territory')
fig.update_layout(
    title='Pie chart visualization of states effected by caronavirus')
fig.show()

In [0]:
#Latitude and Longitude of Indian State
df_with_utm = pd.read_csv("/content/drive/My Drive/epidemic_modelling/UTM ZONES of INDIA.csv")
df_with_utm

In [0]:
#Data of Population,Cases of Carona and UTM 
df_pop_coro_utm = df_population_with_corona_case.merge(df_with_utm , left_on='State / Union Territory', right_on='State / Union Territory')
df_pop_coro_utm

In [0]:
#Total Case in Indian States
fig = go.Figure(data=[go.Scatter(
    x=df_pop_coro_utm['State / Union Territory'][0:36],
    y=df_pop_coro_utm['Total Cases'][0:36],
    mode='markers',
    marker=dict(
        size=[100,90,80, 70, 60, 50, 40,35,35,35,35,35,35,35,35,35,35,30,28,28,25,25,20,15,15,15,15,10,10,10,10],
        showscale=True
        )
)])
fig.update_layout(
    title='Total Case in Indian States',
    xaxis_title="States",
    yaxis_title="Total Cases",
)
fig.show()

In [0]:
#Active Case in Indian States
fig = go.Figure(data=[go.Scatter(
    x=df_pop_coro_utm['State / Union Territory'][0:36],
    y=df_pop_coro_utm['Active Cases'][0:36],
    mode='markers',
    marker=dict(
        size=[100,90,80, 70, 60, 50, 40,35,35,35,35,35,35,35,35,35,35,30,28,28,25,25,20,15,15,15,15,10,10,10,10],
        showscale=True
        )
)])
fig.update_layout(
    title='Active Case in Indian States',
    xaxis_title="States",
    yaxis_title="Active Cases",
)
fig.show()

In [0]:
#India's Map with Statewise data of Total Cases,Deaths and Cure
India_map = folium.Map(location=[20.5937, 78.9629],zoom_start=4.55)
fg=folium.FeatureGroup(name="my map")
fg.add_child(folium.GeoJson(data=(open('/content/drive/My Drive/epidemic_modelling/states_of_india.json','r',encoding='utf-8-sig').read())))
India_map.add_child(fg)
for lat,lan,name,cured,deaths,cases in zip(df_pop_coro_utm['Latitude'],df_pop_coro_utm['Longitude'],df_pop_coro_utm['State / Union Territory'],df_pop_coro_utm['Cured'],df_pop_coro_utm['Deaths'],df_pop_coro_utm['Total Cases']):
    if(deaths == 0):
        folium.Marker(location=[lat,lan],popup="<b>State  : </b>"+name+ "<br> <b>Total Cases : </b> "+str(cases)+"<br> <b>Deaths : </b> "+str(deaths)+"<br> <b>Cured : </b> "+str(cured)).add_to(India_map)
    else:
        folium.Marker(location=[lat,lan],popup="<b>State  : </b>"+name+ "<br> <b>Total Cases : </b> "+str(cases)+"<br> <b>Deaths : </b> "+str(deaths)+"<br> <b>Cured : </b> "+str(cured),icon=folium.Icon(color="red")).add_to(India_map)
India_map

In [0]:
#Total Cases,Active Cases,Cured,Deaths from Corona Virus in India
corona_data = df_corona_in_india.groupby(['Date'])['Total Cases','Active Cases','Cured','Deaths'].sum().reset_index().sort_values('Date',ascending=False)
fig = go.Figure()
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Total Cases'],
                    mode='lines+markers',name='Total Cases'))
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Active Cases'], 
                mode='lines+markers',name='Active Cases'))
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Cured'], 
                mode='lines+markers',name='Cured'))
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Deaths'], 
                mode='lines+markers',name='Deaths'))
fig.update_layout(title_text='Curve Showing Different Cases from COVID-19 in India',plot_bgcolor='rgb(225,230,255)')
fig.show()

INDIAN COUNCIL OF MEDICAL RESEARCH(ICMR) REPORT TILL 9th APRIL









In [0]:
#Testing till 25 March
df_ICMR ['DateTime'] =pd.to_datetime(df_ICMR .DateTime,dayfirst=True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ICMR['DateTime'], y=df_ICMR['TotalSamplesTested'],
                    mode='lines+markers',name='TotalSamplesTested'))
fig.add_trace(go.Scatter(x=df_ICMR['DateTime'], y=df_ICMR['TotalIndividualsTested'], 
                mode='lines+markers',name='TotalIndividualsTested'))
fig.add_trace(go.Scatter(x=df_ICMR['DateTime'], y=df_ICMR['TotalPositiveCases'], 
                mode='lines+markers',name='TotalPositiveCases'))
fig.update_layout(title_text='ICMR TEST for COVID-19',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
#Current Status of Patient wrt state he/she is QUARTINE and his/her Nationality
df_Individual = df_Individual.fillna({
    'nationality': 'Unknown','current_status': 'Unknown'})
df_Individual_new= df_Individual.drop(labels=['id','government_id','diagnosed_date','age','gender','detected_city','detected_district','status_change_date','notes'],axis=1)
df_Individual_new = df_Individual.groupby(['current_status','nationality','detected_state'])['id'].count().reset_index(name='count')
fig = px.bar(df_Individual_new, x='count', y='detected_state', orientation='h',hover_data =['current_status','nationality','detected_state'], color='current_status',height=700)
fig.update_layout(
    title='Current Status of Patient wrt state he/she is QUARTINE and his/her Nationality')
fig.show()

CORONA CASES in INDIA vs ITALY

In [0]:
#Total Confirmed Cases INDIA vs ITALY
df_Italy_p = pd.read_csv("/content/drive/My Drive/epidemic_modelling/covid19_italy_province.csv")
corona_data_ITALY = df_Italy_p.groupby(['Date'])['TotalPositiveCases'].sum().reset_index().sort_values('TotalPositiveCases',ascending = True)

corona_data_INDIA =df_corona_in_india.groupby(['Date'])['Confirmed'].sum().reset_index().sort_values('Date',ascending = True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=corona_data_ITALY['Date'], y=corona_data_ITALY['TotalPositiveCases'],
                    mode='lines+markers',name='Italy'))
fig.add_trace(go.Scatter(x=corona_data_INDIA['Date'], y=corona_data_INDIA['Confirmed'],
                    mode='lines+markers',name='India'))
fig.update_layout(title_text='Total Confirmed Cases INDIA vs ITALY',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
#COVID-19 Recovered Cases in INDIA Vs ITALY
df_Italy_r = pd.read_csv("/content/drive/My Drive/epidemic_modelling/covid19_italy_region.csv")
df_Italy_r['Total Cases'] = df_Italy_r['HospitalizedPatients']+df_Italy_r['IntensiveCarePatients'] + df_Italy_r['TotalHospitalizedPatients'] + df_Italy_r['HomeConfinement'] + df_Italy_r['CurrentPositiveCases'] + df_Italy_r['NewPositiveCases'] + df_Italy_r['Recovered'] + df_Italy_r['Deaths'] + df_Italy_r['TotalPositiveCases']
corona_data_ITALY = df_Italy_r.groupby(['Date'])['Recovered'].sum().reset_index().sort_values('Date',ascending = True)
df_corona_in_india['Date'] =pd.to_datetime(df_corona_in_india.Date,dayfirst=True)
corona_data_INDIA =df_corona_in_india.groupby(['Date'])['Cured'].sum().reset_index().sort_values('Date',ascending = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=corona_data_ITALY['Date'], y=corona_data_ITALY['Recovered'],
                    mode='lines+markers',name='Italy'))
fig.add_trace(go.Scatter(x=corona_data_INDIA['Date'], y=corona_data_INDIA['Cured'],
                    mode='lines+markers',name='India'))
fig.update_layout(title_text='COVID-19 Recovered Cases in INDIA Vs ITALY',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
#COVID-19 Death Cases in ITALY VS INDIA
df_Italy_r = pd.read_csv("/content/drive/My Drive/epidemic_modelling/covid19_italy_region.csv")
df_Italy_r['Total Cases'] = df_Italy_r['HospitalizedPatients']+df_Italy_r['IntensiveCarePatients'] + df_Italy_r['TotalHospitalizedPatients'] + df_Italy_r['HomeConfinement'] + df_Italy_r['CurrentPositiveCases'] + df_Italy_r['NewPositiveCases'] + df_Italy_r['Recovered'] + df_Italy_r['Deaths'] + df_Italy_r['TotalPositiveCases']
corona_data_ITALY = df_Italy_r.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Date',ascending = True)
df_corona_in_india['Date'] =pd.to_datetime(df_corona_in_india.Date,dayfirst=True)
corona_data_INDIA =df_corona_in_india.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Date',ascending = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=corona_data_ITALY['Date'], y=corona_data_ITALY['Deaths'],
                    mode='lines+markers',name='Italy'))
fig.add_trace(go.Scatter(x=corona_data_INDIA['Date'], y=corona_data_INDIA['Deaths'],
                    mode='lines+markers',name='India'))
fig.update_layout(title_text='COVID-19 Death Cases in INDIA Vs ITALY',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
#Daily Deaths in Italy
oarona_data = df_Italy_r.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Deaths',ascending = True)
corona_data['Daily Deaths'] = corona_data['Deaths'].sub(corona_data['Deaths'].shift())
corona_data['Daily Deaths'].iloc[0] = corona_data['Deaths'].iloc[0]
corona_data['Daily Deaths'] = corona_data['Daily Deaths'].astype(int)
fig = px.bar(corona_data, y='Daily Deaths', x='Date',hover_data =['Daily Deaths'], color='Daily Deaths', height=500)
fig.update_layout(
    title='DAILY DEATHS IN ITALY')
fig.show()

CORONA CASES in INDIA vs USA

In [0]:
#Total Corona Cases INDIA vs USA
corona_data_INDIA =df_corona_in_india.groupby(['Date'])['Total Cases'].sum().reset_index().sort_values('Date',ascending = True)
corona_data_USA= df_daily_US.groupby(['dateChecked'])['positive'].sum().reset_index().sort_values('dateChecked',ascending = True)
fig = go.Figure()

fig.add_trace(go.Scatter(x=corona_data_USA['dateChecked'], y=corona_data_USA['positive'],
                    mode='lines+markers',name='USA'))

fig.add_trace(go.Scatter(x=corona_data_INDIA['Date'], y=corona_data_INDIA['Total Cases'],
                    mode='lines+markers',name='India'))

fig.update_layout(title_text='Total Corona Cases INDIA vs USA',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
#COVID-19 Recovered Cases in INDIA vs USA
corona_data_INDIA =df_corona_in_india.groupby(['Date'])['Cured'].sum().reset_index().sort_values('Date',ascending = True)
corona_data_USA= df_daily_US.groupby(['dateChecked'])['recovered'].sum().reset_index().sort_values('dateChecked',ascending = True)
fig = go.Figure()

fig.add_trace(go.Scatter(x=corona_data_USA['dateChecked'], y=corona_data_USA['recovered'],
                    mode='lines+markers',name='USA'))

fig.add_trace(go.Scatter(x=corona_data_INDIA['Date'], y=corona_data_INDIA['Cured'],
                    mode='lines+markers',name='India'))

fig.update_layout(title_text='COVID-19 Recovered Cases in INDIA vs USA',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
#COVID-19 Death Cases in INDIA vs USA
corona_data_INDIA =df_corona_in_india.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Date',ascending = True)
corona_data_USA= df_daily_US.groupby(['dateChecked'])['death'].sum().reset_index().sort_values('dateChecked',ascending = True)
fig = go.Figure()

fig.add_trace(go.Scatter(x=corona_data_USA['dateChecked'], y=corona_data_USA['death'],
                    mode='lines+markers',name='USA'))

fig.add_trace(go.Scatter(x=corona_data_INDIA['Date'], y=corona_data_INDIA['Deaths'],
                    mode='lines+markers',name='India'))

fig.update_layout(title_text='COVID-19 Death Cases in INDIA vs USA',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
#DAILY DEATHS IN USA
corona_data = df_daily_US.groupby(['dateChecked'])['death'].sum().reset_index().sort_values('death',ascending = True)
corona_data['DAILY DEATHS'] = corona_data['death'].sub(corona_data['death'].shift())
corona_data['DAILY DEATHS'].iloc[0] = corona_data['death'].iloc[0]
corona_data['DAILY DEATHS'] = corona_data['DAILY DEATHS'].astype(int)
fig = px.bar(corona_data, y='DAILY DEATHS', x='dateChecked',hover_data =['DAILY DEATHS'], color='DAILY DEATHS', height=500)
fig.update_layout(
    title='DAILY DEATHS IN USA')
fig.show()

In [0]:
#Total Confirmed Cases ITALY vs USA
df_Italy_p = pd.read_csv("/content/drive/My Drive/epidemic_modelling/covid19_italy_province.csv")
corona_data_ITALY = df_Italy_p.groupby(['Date'])['TotalPositiveCases'].sum().reset_index().sort_values('TotalPositiveCases',ascending = True)

#carona_data_INDIA =df_carona_in_india.groupby(['Date'])['Confirmed'].sum().reset_index().sort_values('Date',ascending = True)
corona_data_USA= df_daily_US.groupby(['dateChecked'])['positive'].sum().reset_index().sort_values('dateChecked',ascending = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=corona_data_ITALY['Date'], y=corona_data_ITALY['TotalPositiveCases'],
                    mode='lines+markers',name='Italy'))
fig.add_trace(go.Scatter(x=corona_data_USA['dateChecked'], y=corona_data_USA['positive'],
                    mode='lines+markers',name='usa'))
fig.update_layout(title_text='Total Confirmed Cases ITALY vs USA',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
#COVID-19 Recovered Cases in ITALY VS USA

corona_data_USA= df_daily_US.groupby(['dateChecked'])['recovered'].sum().reset_index().sort_values('dateChecked',ascending = True)
corona_data_ITALY = df_Italy_r.groupby(['Date'])['Recovered'].sum().reset_index().sort_values('Date',ascending = True)
fig = go.Figure()

fig.add_trace(go.Scatter(x=corona_data_USA['dateChecked'], y=corona_data_USA['recovered'],
                    mode='lines+markers',name='USA'))


fig.add_trace(go.Scatter(x=corona_data_ITALY['Date'], y=corona_data_ITALY['Recovered'],
                   mode='lines+markers',name='ITALY'))

fig.update_layout(title_text='COVID-19 Recovered Cases in USA VS ITALY',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
#COVID-19 Death Cases in ITALY VS USA


corona_data_ITALY = df_Italy_r.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Date',ascending = True)
corona_data_USA= df_daily_US.groupby(['dateChecked'])['death'].sum().reset_index().sort_values('dateChecked',ascending = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=corona_data_ITALY['Date'], y=corona_data_ITALY['Deaths'],
                    mode='lines+markers',name='Italy'))

fig.add_trace(go.Scatter(x=corona_data_USA['dateChecked'], y=corona_data_USA['death'],
                    mode='lines+markers',name='USA'))
fig.update_layout(title_text='COVID-19 Death Cases in ITALY VS USA',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [0]:
corona_data_INDIA =df_corona_in_india.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Date',ascending = True)
corona_data_ITALY = df_Italy_r.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Date',ascending = True)
corona_data_USA= df_daily_US.groupby(['dateChecked'])['death'].sum().reset_index().sort_values('dateChecked',ascending = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=corona_data_ITALY['Date'], y=corona_data_ITALY['Deaths'],
                    mode='lines+markers',name='Italy'))

fig.add_trace(go.Scatter(x=corona_data_USA['dateChecked'], y=corona_data_USA['death'],
                    mode='lines+markers',name='USA'))
fig.add_trace(go.Scatter(x=corona_data_INDIA['Date'], y=corona_data_INDIA['Deaths'],
                    mode='lines+markers',name='India'))
fig.update_layout(title_text='COVID-19 Death Cases in ITALY VS USA',plot_bgcolor='rgb(225,230,255)')
fig.show()
